In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import datetime
import pandas as pd
import numpy as np
from functools import partial

from utils.datafetch import *
from utils.vectorized_funs import *
from utils.datapipe import *

> We'll need some way of loading our possible tickets.

> We'll have some information for AMEX, NASDAQ and NYSE on one csv each in the config folder

In [4]:
rr = load_exchangesinfos()

amex has 400 tickers.
nasdaq has 3081 tickers.
nyse has 3299 tickers.


Final dataset has 6780 records
(NOK) Final dataset has duplicate tickers (6779 distinct)



> We'll look into the duplicate tickers when applicable

In [5]:
_r = duplicate_tickers(rr, ign_exchange=True)
_r

,Symbol,Name,MarketCap,Sector,Industry,exchange,ignoring_exchange
2716,MSG,The Madison Square Garden Company,5.837670e+09,Consumer Services,Television Services,nasdaq,False
2922,MSG,The Madison Square Garden Company,5.837670e+09,Consumer Services,Television Services,nyse,True


> in this case the ticker is the same company appearing in two exchanges, so we'll drop one of them

In [6]:
rr = rr.drop([2922])

In [7]:
## this time this should be empty...

duplicate_tickers(rr, ign_exchange=True)

,Symbol,Name,MarketCap,Sector,Industry,exchange,ignoring_exchange


> lets have a look at your datareader wrapper

In [8]:
fetch_quotes("AMZN", from_date=datetime.datetime(2016,12,1))

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-12-01,752.409973,753.369995,738.030029,743.650024,4666000,743.650024
2016-12-02,743.400024,748.489990,736.700012,740.340027,3561300,740.340027
2016-12-05,745.000000,761.489990,742.000000,759.359985,4314700,759.359985
2016-12-06,763.989990,768.239990,757.250000,764.719971,3794700,764.719971
2016-12-07,764.549988,770.419983,755.820007,770.419983,3684900,770.419983
2016-12-08,771.869995,773.789978,765.190002,767.330017,3189600,767.330017
2016-12-09,770.000000,770.250000,765.340027,768.659973,2470900,768.659973
2016-12-12,766.400024,766.890015,757.200012,760.119995,2963900,760.119995
2016-12-13,764.960022,782.460022,762.000000,774.340027,5285300,774.340027


> Lets see how we could prepare an initial data load

In [11]:
tkrs = dict()
errs = dict()
itr = None

for tkr in rr.iloc[0:5]["Symbol"].values.tolist():
    itr = fetch_quotes(tkr)
    if itr is None:
        errs[tkr] = itr
        print("ERROR for {}".format(tkr))
    else:
        tkrs[tkr] = itr
        print("Received {}".format(tkr))


Received XXII
Received FAX
Received IAF
Received CH
ERROR for ABE           


In [9]:
_r = initial_dataload(["AAPL", "TWTR", "XXII"])

(1/3) Recv. and Stored AAPL
(2/3) Recv. and Stored TWTR
(3/3) Recv. and Stored XXII
Took 0:00:00.882025


> lets try with the ticker dataframe now

In [11]:
OKs = ["OK" for x in _r["OK"]]
NOKs = ["NOK" for x in _r["NOK"]]

o = pd.DataFrame([OKs, _r["OK"]]).T
n = pd.DataFrame([NOKs, _r["NOK"]]).T

d = pd.concat([o, n], axis=0)
d.columns = ["STATUS", "Symbol"]
d.to_csv("RES_initial_dataload.csv", index=False)

d

,STATUS,Symbol
0,OK,AAPL
1,OK,TWTR
2,OK,XXII


In [13]:
ticker_list = rr.iloc[0:100]["Symbol"].values
#ticker_list = rr["Symbol"].values
_r = initial_dataload(ticker_list, verbose=True, del_temp=True)
ticker_list

(1/100) Recv. and Stored XXII
(2/100) Recv. and Stored FAX
(3/100) Recv. and Stored IAF
(4/100) Recv. and Stored CH
(5/100) ERROR receiving ABE           
(6/100) Recv. and Stored FCO
(7/100) Recv. and Stored IF
(8/100) Recv. and Stored ISL
(9/100) Recv. and Stored ACU
(10/100) Recv. and Stored AIII
(11/100) Recv. and Stored ATNM
(12/100) Recv. and Stored AE
(13/100) Recv. and Stored ADK
(14/100) ERROR receiving ADK^A
(15/100) Recv. and Stored ACY
(16/100) Recv. and Stored WGA
(17/100) Recv. and Stored AIRI
(18/100) ERROR receiving AA^
(19/100) ERROR receiving AXX
(20/100) Recv. and Stored AXU
(21/100) Recv. and Stored AAU
(22/100) Recv. and Stored APT
(23/100) ERROR receiving ALTV
(24/100) Recv. and Stored AAMC
(25/100) Recv. and Stored DIT
(26/100) ERROR receiving APP
(27/100) Recv. and Stored ADGE
(28/100) Recv. and Stored ALN
(29/100) Recv. and Stored AMS
(30/100) Recv. and Stored AMPE
(31/100) Recv. and Stored AXN
(32/100) Recv. and Stored AMCO
(33/100) Recv. and Stored HRT
(34/10

array(['XXII', 'FAX', 'IAF', 'CH', 'ABE           ', 'FCO', 'IF', 'ISL',
       'ACU', 'AIII', 'ATNM', 'AE', 'ADK', 'ADK^A', 'ACY', 'WGA', 'AIRI',
       'AA^', 'AXX', 'AXU', 'AAU', 'APT', 'ALTV', 'AAMC', 'DIT', 'APP',
       'ADGE', 'ALN', 'AMS', 'AMPE', 'AXN', 'AMCO', 'HRT', 'AKG', 'AINC',
       'ASB.WS', 'AST', 'AWX', 'AVL', 'ASM', 'BTG', 'BTN', 'BKJ', 'BCV',
       'BAA', 'BHB', 'BRN', 'BIR^A', 'BGSF', 'BPMX', 'BTX', 'BTX.WS',
       'BGI', 'BZM', 'MHE', 'BLE', 'BLJ', 'BFY', 'BHV', 'BDR', 'BRG',
       'BVX', 'BWL.A', 'BZC', 'BTI', 'CANF', 'ROX', 'CAW', 'CVM', 'CVM.WS',
       'CEF', 'GTU', 'CET', 'LEU', 'CCF', 'CQH', 'CQP', 'LNG', 'CVR',
       'CNR', 'CPHI', 'CKX', 'GLV', 'GLQ', 'GLO', 'CRV', 'MOC', 'CIX',
       'LODE', 'CTO', 'MCF', 'CUO', 'CMT', 'CVRS', 'CRMD', 'CRF', 'CLM',
       'CVU', 'CIK', 'DHY'], dtype=object)

^ It is nicer to have the time from start to finish of this

In [ ]:
start = datetime.datetime.now()

In [ ]:
end = datetime.datetime.now()

In [ ]:
d = end-start

In [ ]:
d.seconds

In [ ]:
print(str(d))

> Let's try to read back in one of the files we just created....

In [ ]:
XXII = pd.read_csv("{}/{}.csv".format(DATA_PATH, "XXII"))

print("Shape: {}".format(XXII.shape))
print("\nStats\n")
print(XXII.describe())

> Lets see what happens if the file does not exist....
> We might want to consider the FileNotFoundError...

In [ ]:
try:
    XXIIAAAAAAAAA = pd.read_csv("{}/{}.csv".format(DATA_PATH, "XXIIAAAAAAAAA"))
except (FileNotFoundError):
    print("File not found.")
    print("We could create a function to download the data instead of erroring...")

> While we're here, let's look at the diference between csv and hdfs...

In [73]:
_tkrs = ["AAPL", "TWTR", "XXII", "AMZN", "GOOG"]

_r = initial_dataload(_tkrs)

for t in _tkrs:
    itr = pd.read_csv("{}/{}.csv".format(DATA_PATH, t))
    itr.to_hdf("{}/{}.h5".format(DATA_PATH, t), "stock_daily")

(1/5) Recv. and Stored AAPL
(2/5) Recv. and Stored TWTR
(3/5) Recv. and Stored XXII
(4/5) Recv. and Stored AMZN
(5/5) Recv. and Stored GOOG
Took 0:00:01.538005


In [74]:
%%bash
cd data
pwd
ls -lh
du -h

/home/jovyan/work/FintechCapstone/data
total 12M
-rw-r--r-- 1 jovyan users 620K Mar 15 14:51 AAPL.csv
-rw-r--r-- 1 jovyan users 3.9M Mar 15 14:51 AAPL.h5
-rw-r--r-- 1 jovyan users 342K Mar 15 14:51 AMZN.csv
-rw-r--r-- 1 jovyan users 2.0M Mar 15 14:51 AMZN.h5
drwxr-xr-x 2 jovyan users 4.0K Feb 17 14:58 B_TIER1
-rw-r--r-- 1 jovyan users 256K Mar 15 14:51 GOOG.csv
-rw-r--r-- 1 jovyan users 1.7M Mar 15 14:51 GOOG.h5
-rw-r--r-- 1 jovyan users  58K Mar 15 14:51 TWTR.csv
-rw-r--r-- 1 jovyan users 1.3M Mar 15 14:51 TWTR.h5
-rw-r--r-- 1 jovyan users  66K Mar 15 14:51 XXII.csv
-rw-r--r-- 1 jovyan users 1.6M Mar 15 14:51 XXII.h5
14M	./B_TIER1
26M	.


> Ok, so hdf5 will help us manage really large files. But is a bit of overkill for this kind of problem right now

In [75]:
_stock = dict()

_stock["AAPL"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "AAPL"))
_stock["TWTR"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "TWTR"))
_stock["XXII"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "XXII"))
_stock["GOOG"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "GOOG"))
_stock["AMZN"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "AMZN"))

> Now, lets settle on a little ticker list

In [76]:
_tkrs_df = pd.DataFrame(_tkrs, columns=["ticker"])
_tkrs_df

,ticker
0,AAPL
1,TWTR
2,XXII
3,AMZN
4,GOOG


> Let us take a look at their min and max dates

In [77]:
_tkrs_df["min"] = _tkrs_df["ticker"].apply(lambda s: _stock[s]["Date"].min())
_tkrs_df["max"] = _tkrs_df["ticker"].apply(lambda s: _stock[s]["Date"].max())

_tkrs_df

,ticker,min,max
0,AAPL,1980-12-12,2017-03-14
1,TWTR,2013-11-07,2017-03-14
2,XXII,2011-01-26,2017-03-14
3,AMZN,1997-05-15,2017-03-14
4,GOOG,2004-08-19,2017-03-14


> We have been ignoring the fact that the dates come as strings from the csv

In [78]:
_stock["GOOG"]["Date"].head()

0    2004-08-19
1    2004-08-20
2    2004-08-23
3    2004-08-24
4    2004-08-25
Name: Date, dtype: object

> Notice the **dtype** is "object" instead of "date".

> We can take care of this using **to_datetime**

In [79]:
_stock["GOOG"].iloc[3142:3200]["Date"]

3142    2017-02-10
3143    2017-02-13
3144    2017-02-14
3145    2017-02-15
3146    2017-02-16
3147    2017-02-17
3148    2017-02-21
3149    2017-02-22
3150    2017-02-23
3151    2017-02-24
3152    2017-02-27
3153    2017-02-28
3154    2017-03-01
3155    2017-03-02
3156    2017-03-03
3157    2017-03-06
3158    2017-03-07
3159    2017-03-08
3160    2017-03-09
3161    2017-03-10
3162    2017-03-13
3163    2017-03-14
Name: Date, dtype: object

In [80]:
pd.to_datetime(_stock["GOOG"].iloc[:3142]["Date"], infer_datetime_format=True).head()

0   2004-08-19
1   2004-08-20
2   2004-08-23
3   2004-08-24
4   2004-08-25
Name: Date, dtype: datetime64[ns]

In [81]:
def convert_dates(s):
    _stock[s]["Date"] = pd.to_datetime(_stock[s]["Date"], infer_datetime_format=True)

_a = _tkrs_df["ticker"].apply(convert_dates)

print(_stock["GOOG"]["Date"].head())
print("\n")
print(_stock["AAPL"]["Date"].head())

0   2004-08-19
1   2004-08-20
2   2004-08-23
3   2004-08-24
4   2004-08-25
Name: Date, dtype: datetime64[ns]


0   1980-12-12
1   1980-12-15
2   1980-12-16
3   1980-12-17
4   1980-12-18
Name: Date, dtype: datetime64[ns]


In [82]:
_stock["GOOG"].head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2004-08-19,100.000168,104.060182,95.960165,100.340176,44871300,50.119968
1,2004-08-20,101.010175,109.080187,100.500174,108.310183,22942800,54.100990
2,2004-08-23,110.750191,113.480193,109.050183,109.400185,18342800,54.645447
3,2004-08-24,111.240189,111.600192,103.570177,104.870176,15319700,52.382705
4,2004-08-25,104.960181,108.000187,103.880180,106.000184,9232100,52.947145


In [83]:
_tkrs_df

,ticker,min,max
0,AAPL,1980-12-12,2017-03-14
1,TWTR,2013-11-07,2017-03-14
2,XXII,2011-01-26,2017-03-14
3,AMZN,1997-05-15,2017-03-14
4,GOOG,2004-08-19,2017-03-14


> From now on it is safe to slice the *Date* columns using datetime objects

In [84]:
_stock["GOOG"]["Date"].head() > datetime.datetime(2004, 8, 20)

0    False
1    False
2     True
3     True
4     True
Name: Date, dtype: bool

In [85]:
_stock["GOOG"]["Date"].head() > datetime.datetime(2004, 8, 24)

0    False
1    False
2    False
3    False
4     True
Name: Date, dtype: bool

> Lets take a look at rolling indicators

In [86]:
pd.rolling_mean(_stock["GOOG"]["Open"], 5).iloc[0:14]

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).mean()
  if __name__ == '__main__':


0            NaN
1            NaN
2            NaN
3            NaN
4     105.592181
5     106.582183
6     108.000185
7     106.906183
8     105.118179
9     104.666178
10    103.514177
11    102.084174
12    101.230174
13    100.918174
Name: Open, dtype: float64

In [87]:
_stock["GOOG"]["Open"].rolling(window=5, center=False).mean().iloc[0:14]

0            NaN
1            NaN
2            NaN
3            NaN
4     105.592181
5     106.582183
6     108.000185
7     106.906183
8     105.118179
9     104.666178
10    103.514177
11    102.084174
12    101.230174
13    100.918174
Name: Open, dtype: float64

In [88]:
_stock["GOOG"]["Close"].rolling(window=5, center=False).mean().iloc[0:14]

0            NaN
1            NaN
2            NaN
3            NaN
4     105.784181
5     107.298182
6     106.866182
7     105.388179
8     104.888179
9     103.738176
10    102.458174
11    101.230172
12    101.144173
13    101.130172
Name: Close, dtype: float64

> One can slice a subset of columns using loc and then call apply across all the sliced columns.

> The result will be a new dataframe with the rolled averages row-by-row across each column

In [89]:
def apply_rolling_mean(s):
    return s.rolling(window=5, center=False).mean()

_rolling5 = _stock["GOOG"].loc[:, ["Open", "High", "Low", "Close", "Volume", "Adj Close"]].apply(apply_rolling_mean)
_rolling5.iloc[0:14]


,Open,High,Low,Close,Volume,Adj Close
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,105.592181,109.244188,102.592176,105.784181,22141740.0,52.839251
5,106.582183,110.022189,104.332179,107.298182,14593200.0,53.595495
6,108.000185,109.930189,105.370180,106.866182,11252880.0,53.379711
7,106.906183,108.332187,103.962178,105.388179,8628600.0,52.641448
8,105.118179,106.754185,103.680178,104.888179,6552900.0,52.391698
9,104.666178,105.748184,102.838175,103.738176,6542800.0,51.817271


> We can concatenate the new frame and the old one using concat (we need to set the *axis=* in order to do it across columns)

> We can also reset the column names so that we don't have duplicate names (we could have done this in the previous step on the _rolling5 frame)

In [90]:
_result = pd.concat([_stock["GOOG"], _rolling5], axis=1)
_result.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Open_roll5','High_roll5', 'Low_roll5', 'Close_roll5', 'Volume_roll5', 'Adj Close_roll5']

_result.iloc[0:14]

,Date,Open,High,Low,Close,Volume,Adj Close,Open_roll5,High_roll5,Low_roll5,Close_roll5,Volume_roll5,Adj Close_roll5
0,2004-08-19,100.000168,104.060182,95.960165,100.340176,44871300,50.119968,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-08-20,101.010175,109.080187,100.500174,108.310183,22942800,54.100990,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-08-23,110.750191,113.480193,109.050183,109.400185,18342800,54.645447,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-08-24,111.240189,111.600192,103.570177,104.870176,15319700,52.382705,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-08-25,104.960181,108.000187,103.880180,106.000184,9232100,52.947145,105.592181,109.244188,102.592176,105.784181,22141740.0,52.839251
5,2004-08-26,104.950180,107.950188,104.660179,107.910182,7128600,53.901190,106.582183,110.022189,104.332179,107.298182,14593200.0,53.595495
6,2004-08-27,108.100185,108.620186,105.690180,106.150181,6241200,53.022069,108.000185,109.930189,105.370180,106.866182,11252880.0,53.379711
7,2004-08-30,105.280178,105.490184,102.010172,102.010172,5221400,50.954132,106.906183,108.332187,103.962178,105.388179,8628600.0,52.641448
8,2004-08-31,102.300173,103.710180,102.160177,102.370175,4941200,51.133953,105.118179,106.754185,103.680178,104.888179,6552900.0,52.391698
9,2004-09-01,102.700174,102.970180,99.670169,100.250171,9181600,50.075011,104.666178,105.748184,102.838175,103.738176,6542800.0,51.817271


> Lets see how we could repurpose **apply_rolling_mean** to use with different window sizes with an apply over columns...

In [91]:
def apply_rolling_mean(s, window=5):
    return s.rolling(window=window, center=False).mean()

_col_slice = ["Open", "High", "Low", "Close", "Volume", "Adj Close"]
_pre_bound = partial(apply_rolling_mean, window=3)
_rolling = _stock["GOOG"].loc[ : , _col_slice ].apply(_pre_bound)
_rolling.iloc[0:8]


,Open,High,Low,Close,Volume,Adj Close
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,103.920178,108.873521,101.836841,106.016848,2.871897e+07,52.955468
3,107.666852,111.386857,104.373511,107.526848,1.886843e+07,53.709714
4,108.983520,111.026857,105.500180,106.756848,1.429820e+07,53.325099
5,107.050183,109.183522,104.036845,106.260181,1.056013e+07,53.077013
6,106.003515,108.190187,104.743513,106.686849,7.533967e+06,53.290135
7,106.110181,107.353519,104.120177,105.356845,6.197067e+06,52.625797


In [92]:
# We can also call rolling directly from the dataframe itself
_stock["GOOG"].loc[:,["Open"]].rolling(window=3, center=False).mean().head()

,Open
0,NaN
1,NaN
2,103.920178
3,107.666852
4,108.983520


In [93]:
# this will be the same as the cells above
a = _stock["GOOG"].loc[:,["Open"]].rolling(window=3, center=False)
getattr(a, 'mean')().head()

,Open
0,NaN
1,NaN
2,103.920178
3,107.666852
4,108.983520


In [94]:
# but now we can call other rolling stats programatically

stat_fns = ["mean", "sum"]

for itr in stat_fns:
    print("{} :\n".format(itr))
    print(getattr(a, itr)().head())
    print("\n---\n")

mean :

         Open
0         NaN
1         NaN
2  103.920178
3  107.666852
4  108.983520

---

sum :

         Open
0         NaN
1         NaN
2  311.760534
3  323.000555
4  326.950561

---



> We can bundle all this rolling logic in a function that rolls and merges the collumns that we want

In [95]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open"]).iloc[0:16]

,Open_roll_mean_10
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,105.129179


In [96]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"]).iloc[0:16]

,Open_roll_mean_10,Close_roll_mean_10
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,105.129179,104.761178


In [97]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=5).head()

,Open_roll_mean_5,Close_roll_mean_5
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,105.592181,105.784181


In [98]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=3).head()

,Open_roll_mean_3,Close_roll_mean_3
0,NaN,NaN
1,NaN,NaN
2,103.920178,106.016848
3,107.666852,107.526848
4,108.983520,106.756848


In [99]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=3, merge_result=True).head()

,Date,Open,High,Low,Close,Volume,Adj Close,Open_roll_mean_3,Close_roll_mean_3
0,2004-08-19,100.000168,104.060182,95.960165,100.340176,44871300,50.119968,NaN,NaN
1,2004-08-20,101.010175,109.080187,100.500174,108.310183,22942800,54.100990,NaN,NaN
2,2004-08-23,110.750191,113.480193,109.050183,109.400185,18342800,54.645447,103.920178,106.016848
3,2004-08-24,111.240189,111.600192,103.570177,104.870176,15319700,52.382705,107.666852,107.526848
4,2004-08-25,104.960181,108.000187,103.880180,106.000184,9232100,52.947145,108.983520,106.756848


> We really shouldn't leave the nan or other missing values there.

In [100]:
df_topad = roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=3).head()
df_topad

,Open_roll_mean_3,Close_roll_mean_3
0,NaN,NaN
1,NaN,NaN
2,103.920178,106.016848
3,107.666852,107.526848
4,108.983520,106.756848


In [101]:
df_topad.fillna(method="bfill")

,Open_roll_mean_3,Close_roll_mean_3
0,103.920178,106.016848
1,103.920178,106.016848
2,103.920178,106.016848
3,107.666852,107.526848
4,108.983520,106.756848


> we could acomodate this on our function....

In [102]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=3, pad_result=False).head()

,Open_roll_mean_3,Close_roll_mean_3
0,NaN,NaN
1,NaN,NaN
2,103.920178,106.016848
3,107.666852,107.526848
4,108.983520,106.756848


In [103]:
roll_columns(_stock["GOOG"], "mean", column_slice=["Open", "Close"], window=3, pad_result=True).head()

,Open_roll_mean_3,Close_roll_mean_3
0,103.920178,106.016848
1,103.920178,106.016848
2,103.920178,106.016848
3,107.666852,107.526848
4,108.983520,106.756848


We can look at how to compute the daily returns by shifting our data

In [104]:
_sample_goog_close = _stock["GOOG"].loc[:,["Close"]].iloc[:15]
_sample_goog_close

,Close
0,100.340176
1,108.310183
2,109.400185
3,104.870176
4,106.000184
5,107.910182
6,106.150181
7,102.010172
8,102.370175
9,100.250171


In [105]:
%%timeit

_rr = (_sample_goog_close[1:] / _sample_goog_close[:-1].values) - 1
_rr.ix[0] = 0
_rr



100 loops, best of 3: 4.05 ms per loop


In [106]:
%%timeit

# this code does the same as the cell above, but as we can see it is much faster
_rr = (_sample_goog_close / _sample_goog_close.shift(1)) - 1
_rr.ix[0] = 0
_rr

1000 loops, best of 3: 1.14 ms per loop


> Lets try with multiple collumns

In [107]:

_rr = (_sample_goog_close / _sample_goog_close.shift(1)) - 1
_rr.ix[0] = 0
_rr

,Close
0,0.000000
1,0.079430
2,0.010064
3,-0.041408
4,0.010775
5,0.018019
6,-0.016310
7,-0.039001
8,0.003529
9,-0.020709


In [108]:
_rr = (_sample_goog_close / _sample_goog_close.shift(1)) - 1
_rr = _rr.fillna(0)
_rr

,Close
0,0.000000
1,0.079430
2,0.010064
3,-0.041408
4,0.010775
5,0.018019
6,-0.016310
7,-0.039001
8,0.003529
9,-0.020709


> Lets try out some functions of our own. Built on top of this

In [109]:
import numpy as np
df = pd.DataFrame([[1, 1.2, 1.8],[1, 0.8, 1.6],[1,-0.5,0.1]]).T
df

,0,1,2
0,1.0,1.0,1.0
1,1.2,0.8,-0.5
2,1.8,1.6,0.1


In [110]:
timewindow_diff(df)

,timewindow_diff_1_0,timewindow_diff_1_1,timewindow_diff_1_2
0,NaN,NaN,NaN
1,0.2,-0.2,-1.5
2,0.6,0.8,0.6


In [111]:
timewindow_diff(df, shift_window=2, fillna=True)

,timewindow_diff_2_0,timewindow_diff_2_1,timewindow_diff_2_2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.8,0.6,-0.9


In [112]:
timewindow_diff(df, column_slice=[1], fillna=True, merge_result=True)

,0,1,2,timewindow_diff_1_1
0,1.0,1.0,1.0,0.0
1,1.2,0.8,-0.5,-0.2
2,1.8,1.6,0.1,0.8


In [113]:
df

,0,1,2
0,1.0,1.0,1.0
1,1.2,0.8,-0.5
2,1.8,1.6,0.1


In [114]:
timewindow_return(df)

,timewindow_return_1_0,timewindow_return_1_1,timewindow_return_1_2
0,NaN,NaN,NaN
1,0.2,-0.2,-1.5
2,0.5,1.0,-1.2


> timewindow_return gives us the variation from an initial value. See the explanation below.

In [115]:
n = 200
p = 1.5
v = p * n
print("You buy {} shares at {}. Your value is {}".format(n,p,v))
np = 1.0
nv = np * n
r = ((np / p) - 1.0)
print("The price changes to {}. You new value is {}.".format(np, nv))

print ("Your return is {} ({} of your initial value)".format(v*r, r))

You buy 200 shares at 1.5. Your value is 300.0
The price changes to 1.0. You new value is 200.0.
Your return is -100.00000000000001 (-0.33333333333333337 of your initial value)


In [116]:
df

,0,1,2
0,1.0,1.0,1.0
1,1.2,0.8,-0.5
2,1.8,1.6,0.1


In [117]:
timewindow_cumdiff(df)

,timewindow_cumdiff_0,timewindow_cumdiff_1,timewindow_cumdiff_2
0,0.0,0.0,0.0
1,0.2,-0.2,-1.5
2,0.8,0.6,-0.9


In [118]:
timewindow_cumreturn(df)

,timewindow_cumreturn_0,timewindow_cumreturn_1,timewindow_cumreturn_2
0,0.0,0.0,0.0
1,0.2,-0.2,-1.5
2,0.8,0.6,-0.9


> Now, lets take a real look at dates and how to manage them

In [119]:
import numpy as np

# This can create an array with a date for us
ledate = np.array('2015-12-30', dtype=np.datetime64)
ledate

array(datetime.date(2015, 12, 30), dtype='datetime64[D]')

> we can obviously do vectorized operation on *ledate*

In [120]:
lerange_12day = ledate + np.arange(12)
lerange_12day

array(['2015-12-30', '2015-12-31', '2016-01-01', '2016-01-02',
       '2016-01-03', '2016-01-04', '2016-01-05', '2016-01-06',
       '2016-01-07', '2016-01-08', '2016-01-09', '2016-01-10'], dtype='datetime64[D]')

> We can also use *np.datetime64* in order to create a date constant. It's possible to give it a time format

In [121]:
print(np.datetime64('2015-07-04 12:00'))
print(np.datetime64('2015-07-04 12:59:59.50', 'ns'))

2015-07-04T12:00
2015-07-04T12:59:59.500000000


> These are the code formats possible with numpy

|Code    | Meaning     | Time span (relative) | Time span (absolute)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

> We can also use pandas to manipulate our dates

In [122]:
ledate = pd.to_datetime("4th of July, 2015")
ledate

Timestamp('2015-07-04 00:00:00')

In [123]:
ledate.strftime('%A')

'Saturday'

> We can also use vectorized operations 

In [124]:
lerange_12days = ledate + pd.to_timedelta(np.arange(12), 'D')
lerange_12days

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

In [125]:
lerange_12months = ledate + pd.to_timedelta(np.arange(12), 'M')
lerange_12months

DatetimeIndex(['2015-07-04 00:00:00', '2015-08-03 10:29:06',
               '2015-09-02 20:58:12', '2015-10-03 07:27:18',
               '2015-11-02 17:56:24', '2015-12-03 04:25:30',
               '2016-01-02 14:54:36', '2016-02-02 01:23:42',
               '2016-03-03 11:52:48', '2016-04-02 22:21:54',
               '2016-05-03 08:51:00', '2016-06-02 19:20:06'],
              dtype='datetime64[ns]', freq=None)

In [126]:
lerange_12minutes = ledate + pd.to_timedelta(np.arange(12), 'm')
lerange_12minutes

DatetimeIndex(['2015-07-04 00:00:00', '2015-07-04 00:01:00',
               '2015-07-04 00:02:00', '2015-07-04 00:03:00',
               '2015-07-04 00:04:00', '2015-07-04 00:05:00',
               '2015-07-04 00:06:00', '2015-07-04 00:07:00',
               '2015-07-04 00:08:00', '2015-07-04 00:09:00',
               '2015-07-04 00:10:00', '2015-07-04 00:11:00'],
              dtype='datetime64[ns]', freq=None)

> We can also use *pd.date_range* to achieve this

In [127]:
pd.date_range('2015-07-03', '2015-07-10')

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

In [128]:
pd.date_range('2015-07-03', periods=4)

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06'], dtype='datetime64[ns]', freq='D')

In [129]:
pd.date_range('2015-07-03', periods=8, freq='H')

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

In [130]:
pd.period_range('2015-07', periods=8, freq='M')

PeriodIndex(['2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
             '2016-01', '2016-02'],
            dtype='period[M]', freq='M')

In [131]:
pd.timedelta_range(0, periods=10, freq='H')

TimedeltaIndex(['00:00:00', '01:00:00', '02:00:00', '03:00:00', '04:00:00',
                '05:00:00', '06:00:00', '07:00:00', '08:00:00', '09:00:00'],
               dtype='timedelta64[ns]', freq='H')

#### Frequencies and Offsets
(Taken from [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook))

Fundamental to these Pandas time series tools is the concept of a frequency or date offset.
Just as we saw the ``D`` (day) and ``H`` (hour) codes above, we can use such codes to specify any desired frequency spacing.
The following table summarizes the main codes available:

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

| Code    | Description            | Code    | Description            |
|---------|------------------------|---------|------------------------|
| ``MS``  | Month start            |``BMS``  | Business month start   |
| ``QS``  | Quarter start          |``BQS``  | Business quarter start |
| ``AS``  | Year start             |``BAS``  | Business year start    |

In [132]:
## This will give us 9 spans of 2 and a half hours each
pd.timedelta_range(0, periods=9, freq="2H30T")

TimedeltaIndex(['00:00:00', '02:30:00', '05:00:00', '07:30:00', '10:00:00',
                '12:30:00', '15:00:00', '17:30:00', '20:00:00'],
               dtype='timedelta64[ns]', freq='150T')

In [133]:
aapl = _stock["AAPL"]
aapl.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1980-12-12,28.750000,28.875000,28.750,28.750,117258400,0.425000
1,1980-12-15,27.375001,27.375001,27.250,27.250,43971200,0.402826
2,1980-12-16,25.375000,25.375000,25.250,25.250,26432000,0.373261
3,1980-12-17,25.875000,25.999999,25.875,25.875,21610400,0.382500
4,1980-12-18,26.625000,26.750000,26.625,26.625,18362400,0.393587


In [134]:
aapl.corr()

,Open,High,Low,Close,Volume,Adj Close
Open,1.000000,0.999929,0.999902,0.999826,0.200913,0.603955
High,0.999929,1.000000,0.999882,0.999921,0.203637,0.603304
Low,0.999902,0.999882,1.000000,0.999915,0.196851,0.604890
Close,0.999826,0.999921,0.999915,1.000000,0.200406,0.604161
Volume,0.200913,0.203637,0.196851,0.200406,1.000000,0.013555
Adj Close,0.603955,0.603304,0.604890,0.604161,0.013555,1.000000


In [135]:
goog = _stock["GOOG"]

In [136]:
_rolling = aapl.loc[:100, ["Close"]].rolling(window=10, center=True)

_rolling.corr(other=goog["Open"]).iloc[:16]

,Close
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,-0.492011
6,-0.696666
7,-0.874905
8,-0.850180
9,-0.807468


In [137]:
aapl["Close"].corr(goog["Close"])

0.49542591210516851

> Lets take a look at the exchanges

In [138]:
_tkrs = ["^IXIC", "^GSPC"]

_r = initial_dataload(_tkrs)

_exchanges = dict()

for t in _tkrs:
    _exchanges[t] = pd.read_csv("{}/{}.csv".format(DATA_PATH, t))

(1/2) Recv. and Stored ^IXIC
(2/2) Recv. and Stored ^GSPC
Took 0:00:01.651249


In [139]:
df_beta = pd.DataFrame()

df_beta["IXIC"] = _exchanges["^IXIC"].iloc[-100:, :]["Close"]
df_beta["GOOG"] = _stock["GOOG"].iloc[-100:, :]["Close"]

In [140]:
def calc_beta(df):
	# first column is the market
	X = df.values[:, [0]]
	# prepend a column of ones for the intercept
	X = np.concatenate([np.ones_like(X), X], axis=1)
	# matrix algebra
	b = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(df.values[:, 1:])
	return pd.Series(b[1], df.columns[1:], name='Beta')




In [141]:
df_beta.head()

,IXIC,GOOG
11527,5246.410156,NaN
11528,5241.830078,NaN
11529,5257.399902,NaN
11530,5309.830078,NaN
11531,5283.399902,NaN


In [142]:
_stock["GOOG"].iloc[-100:, :]["Close"]

3064    801.500000
3065    796.969971
3066    799.369995
3067    813.109985
3068    807.669983
3069    799.070007
3070    795.349976
3071    795.369995
3072    784.539978
3073    783.609985
3074    768.700012
3075    762.130005
3076    762.020020
3077    782.520020
3078    790.510010
3079    785.309998
3080    762.559998
3081    754.020020
3082    736.080017
3083    758.489990
3084    764.479980
3085    771.229980
3086    760.539978
3087    769.200012
3088    768.270020
3089    760.989990
3090    761.679993
3091    768.239990
3092    770.840027
3093    758.039978
           ...    
3134    796.789978
3135    795.695007
3136    798.530029
3137    801.489990
3138    801.340027
3139    806.969971
3140    808.380005
3141    809.559998
3142    813.669983
3143    819.239990
3144    820.450012
3145    818.979980
3146    824.159973
3147    828.070007
3148    831.659973
3149    830.760010
3150    831.330017
3151    828.640015
3152    829.280029
3153    823.210022
3154    835.239990
3155    830.

In [143]:
dates = pd.date_range('1995-12-31', periods=480, freq='M', name='Date')
stoks = pd.Index(['s{:04d}'.format(i) for i in range(4000)])
df = pd.DataFrame(np.random.rand(480, 4000), dates, stoks)

In [144]:
df.iloc[:5, :5]


,s0000,s0001,s0002,s0003,s0004
Date,,,,,
1995-12-31,0.439742,0.244586,0.939956,0.793016,0.498755
1996-01-31,0.751395,0.161854,0.319897,0.999553,0.627204
1996-02-29,0.681048,0.976193,0.980843,0.859208,0.268533
1996-03-31,0.684112,0.731804,0.872493,0.846800,0.610690
1996-04-30,0.044781,0.855973,0.631492,0.314742,0.058679


In [145]:
def roll(df, w):
    # stack df.values w-times shifted once at each stack
    roll_array = np.dstack([df.values[i:i+w, :] for i in range(len(df.index) - w + 1)]).T
    # roll_array is now a 3-D array and can be read into
    # a pandas panel object
    panel = pd.Panel(roll_array, 
                     items=df.index[w-1:],
                     major_axis=df.columns,
                     minor_axis=pd.Index(range(w), name='roll'))
    # convert to dataframe and pivot + groupby
    # is now ready for any action normally performed
    # on a groupby object
    return panel.to_frame().unstack().T.groupby(level=0)


In [146]:
def beta(df):
    # first column is the market
    X = df.values[:, [0]]
    # prepend a column of ones for the intercept
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # matrix algebra
    b = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(df.values[:, 1:])
    return pd.Series(b[1], df.columns[1:], name='Beta')


In [147]:
rdf = roll(df, 12)
betas = rdf.apply(beta)


In [148]:
def calc_beta(df):
    np_array = df.values
    m = np_array[:,0] # market returns are column zero from numpy array
    s = np_array[:,1] # stock returns are column one from numpy array
    covariance = np.cov(s,m) # Calculate covariance between stock and market
    beta = covariance[0,1]/covariance[1,1]
    return beta


In [149]:
print(calc_beta(df.iloc[:12, :2]))


-0.098394398325


In [150]:
print(beta(df.iloc[:12, :2]))


s0001   -0.098394
Name: Beta, dtype: float64


In [151]:
betas = rdf.apply(beta)
betas.iloc[:5, :5]


major,s0001,s0002,s0003,s0004,s0005
Date,,,,,
1996-11-30,-0.098394,-0.184887,0.090722,-0.121429,0.550546
1996-12-31,-0.120075,-0.103704,0.140081,-0.142751,0.564142
1997-01-31,-0.132794,0.000154,0.066946,-0.158786,0.473184
1997-02-28,-0.243285,-0.046319,0.077253,-0.071332,0.458667
1997-03-31,-0.268293,-0.094814,0.007051,-0.125453,0.379129


In [152]:
%%timeit 
roll(df, 12).apply(beta)

1 loop, best of 3: 4.88 s per loop


In [153]:
from scipy import stats
import numpy as np
x = np.random.random(10)
y = np.random.random(10)
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)


In [154]:
_stock = dict()
_stock["TWTR"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "TWTR"))

In [155]:
timespan = None

timespan = {
    "short_term": [5]
    ,"medium_term": [30,50,70]
    ,"long_term": [100, 200]
}


In [156]:
gains = None

gains = {
    "low": [0.01]
    ,"medium": [0.05, 0.07, 0.1]
    ,"high": [0.2, 0.4, 1.0]
}

In [157]:
losses = None

losses = {
    "low": [-0.05, -0.1]
    ,"medium": [-0.3, -0.5]
    ,"high": [-0.8, -1.0, -2.0]
}

In [158]:
itr_df = _stock["TWTR"];

itr_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701600,44.900002
1,2013-11-08,45.930000,46.939999,40.689999,41.650002,27925300,41.650002
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113900,42.900002
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316700,41.900002
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688300,42.599998


In [159]:
## calculate diff moves for timespans
## calculate returns for timespans
## calculate moving averages
## calculate bollinger bands

In [160]:
## calculate diff moves for timespans

itr_df = _stock["TWTR"];

for ts_name in timespan:
    for t in timespan[ts_name]:
        itr_df = timewindow_diff(itr_df, column_slice=["Close", "Volume"], shift_window=t, merge_result=True)        

itr_df.head()


,Date,Open,High,Low,Close,Volume,Adj Close,timewindow_diff_5_Close,timewindow_diff_5_Volume,timewindow_diff_30_Close,timewindow_diff_30_Volume,timewindow_diff_50_Close,timewindow_diff_50_Volume,timewindow_diff_70_Close,timewindow_diff_70_Volume,timewindow_diff_100_Close,timewindow_diff_100_Volume,timewindow_diff_200_Close,timewindow_diff_200_Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701600,44.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-11-08,45.930000,46.939999,40.689999,41.650002,27925300,41.650002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113900,42.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316700,41.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688300,42.599998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
## calculate returns for timespans

itr_df = _stock["TWTR"];

for ts_name in timespan:
    for t in timespan[ts_name]:
        itr_df = timewindow_return(itr_df, column_slice=["Close", "Volume"], shift_window=t, merge_result=True)        

itr_df.head()



,Date,Open,High,Low,Close,Volume,Adj Close,timewindow_return_5_Close,timewindow_return_5_Volume,timewindow_return_30_Close,timewindow_return_30_Volume,timewindow_return_50_Close,timewindow_return_50_Volume,timewindow_return_70_Close,timewindow_return_70_Volume,timewindow_return_100_Close,timewindow_return_100_Volume,timewindow_return_200_Close,timewindow_return_200_Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701600,44.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-11-08,45.930000,46.939999,40.689999,41.650002,27925300,41.650002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113900,42.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316700,41.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688300,42.599998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
## calculate moving averages

itr_df = _stock["TWTR"];

for ts_name in timespan:
    for t in timespan[ts_name]:
        itr_df = roll_columns(itr_df, "mean", column_slice=["Close", "Volume"], window=t, merge_result=True)        

itr_df.head()


,Date,Open,High,Low,Close,Volume,Adj Close,Close_roll_mean_5,Volume_roll_mean_5,Close_roll_mean_30,Volume_roll_mean_30,Close_roll_mean_50,Volume_roll_mean_50,Close_roll_mean_70,Volume_roll_mean_70,Close_roll_mean_100,Volume_roll_mean_100,Close_roll_mean_200,Volume_roll_mean_200
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701600,44.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-11-08,45.930000,46.939999,40.689999,41.650002,27925300,41.650002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113900,42.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316700,41.900002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688300,42.599998,42.790001,35349160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
## calculate bollinger bands

itr_df = pd.DataFrame()
res_df = pd.DataFrame()

for ts_name in timespan:
    if ts_name in ["medium_term", "long_term"]:
        for t in timespan[ts_name]:
            itr_df = roll_columns(_stock["TWTR"], "std", column_slice=["Close"], window=t, merge_result=False, scaler=2, pad_result=True)
            upper_band = itr_df.apply(lambda x: s + x)
            lower_band = itr_df.apply(lambda x: s - x)
            
            roll_name = "{}_roll_2std_{}".format("Close", t)
            upper_name = "{}_bollinger_{}_up".format("Close", t)
            lower_name = "{}_bollinger_{}_low".format("Close", t)
            
            res_df[roll_name] = itr_df.iloc[:, 0]
            res_df[upper_name] = upper_band
            res_df[lower_name] = lower_band

            
res_df.head()

NameError: ("name 's' is not defined", 'occurred at index Close_roll_std_30')

In [12]:
_stock = dict()
_stock["TWTR"] = pd.read_csv("{}/{}.csv".format(DATA_PATH, "TWTR"))

In [13]:
aapl = load_raw_frame("AAPL")
aapl.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1980-12-12,28.750000,28.875000,28.750,28.750,117258400,0.425000
1,1980-12-15,27.375001,27.375001,27.250,27.250,43971200,0.402826
2,1980-12-16,25.375000,25.375000,25.250,25.250,26432000,0.373261
3,1980-12-17,25.875000,25.999999,25.875,25.875,21610400,0.382500
4,1980-12-18,26.625000,26.750000,26.625,26.625,18362400,0.393587


In [14]:
timespan = None

timespan = {
    "medium_term": [4,7]
}


In [15]:
aapl = load_raw_frame("AAPL")

In [16]:
calc_diff_moves(aapl, timespan=timespan, column_slice=["Close", "Volume"], fillna=True, merge_result=True)

,Date,Open,High,Low,Close,Volume,Adj Close,timewindow_diff_4_Close,timewindow_diff_4_Volume,timewindow_diff_7_Close,timewindow_diff_7_Volume
0,1980-12-12,28.750000,28.875000,28.750000,28.750000,117258400,0.425000,0.000000,0.0,0.000000,0.0
1,1980-12-15,27.375001,27.375001,27.250000,27.250000,43971200,0.402826,0.000000,0.0,0.000000,0.0
2,1980-12-16,25.375000,25.375000,25.250000,25.250000,26432000,0.373261,0.000000,0.0,0.000000,0.0
3,1980-12-17,25.875000,25.999999,25.875000,25.875000,21610400,0.382500,0.000000,0.0,0.000000,0.0
4,1980-12-18,26.625000,26.750000,26.625000,26.625000,18362400,0.393587,-2.125000,-98896000.0,0.000000,0.0
5,1980-12-19,28.249999,28.374999,28.249999,28.249999,12157600,0.417608,0.999999,-31813600.0,0.000000,0.0
6,1980-12-22,29.625000,29.750000,29.625000,29.625000,9340800,0.437934,4.375000,-17091200.0,0.000000,0.0
7,1980-12-23,30.874999,30.999999,30.874999,30.874999,11737600,0.456413,4.999999,-9872800.0,2.124999,-105520800.0
8,1980-12-24,32.500000,32.624999,32.500000,32.500000,12000800,0.480434,5.875000,-6361600.0,5.250000,-31970400.0
9,1980-12-26,35.500001,35.625001,35.500001,35.500001,13893600,0.524782,7.250002,1736000.0,10.250001,-12538400.0


In [173]:
calc_return(aapl, timespan=timespan, column_slice=["Close", "Volume"], merge_result=True)

,Date,Open,High,Low,Close,Volume,Adj Close,timewindow_return_4_Close,timewindow_return_4_Volume,timewindow_return_7_Close,timewindow_return_7_Volume
0,1980-12-12,28.750000,28.875000,28.750000,28.750000,117258400,0.425000,0.000000,0.000000,0.000000,0.000000
1,1980-12-15,27.375001,27.375001,27.250000,27.250000,43971200,0.402826,0.000000,0.000000,0.000000,0.000000
2,1980-12-16,25.375000,25.375000,25.250000,25.250000,26432000,0.373261,0.000000,0.000000,0.000000,0.000000
3,1980-12-17,25.875000,25.999999,25.875000,25.875000,21610400,0.382500,0.000000,0.000000,0.000000,0.000000
4,1980-12-18,26.625000,26.750000,26.625000,26.625000,18362400,0.393587,-0.073913,-0.843402,0.000000,0.000000
5,1980-12-19,28.249999,28.374999,28.249999,28.249999,12157600,0.417608,0.036697,-0.723510,0.000000,0.000000
6,1980-12-22,29.625000,29.750000,29.625000,29.625000,9340800,0.437934,0.173267,-0.646610,0.000000,0.000000
7,1980-12-23,30.874999,30.999999,30.874999,30.874999,11737600,0.456413,0.193237,-0.456854,0.073913,-0.899900
8,1980-12-24,32.500000,32.624999,32.500000,32.500000,12000800,0.480434,0.220657,-0.346447,0.192661,-0.727076
9,1980-12-26,35.500001,35.625001,35.500001,35.500001,13893600,0.524782,0.256637,0.142791,0.405941,-0.474364


In [174]:
calc_sma(aapl, timespan, ["Close", "Volume"], merge_result=True)

,Date,Open,High,Low,Close,Volume,Adj Close,Close_roll_mean_4,Volume_roll_mean_4,Close_roll_mean_7,Volume_roll_mean_7
0,1980-12-12,28.750000,28.875000,28.750000,28.750000,117258400,0.425000,26.781250,52318000.0,27.375000,3.559040e+07
1,1980-12-15,27.375001,27.375001,27.250000,27.250000,43971200,0.402826,26.781250,52318000.0,27.375000,3.559040e+07
2,1980-12-16,25.375000,25.375000,25.250000,25.250000,26432000,0.373261,26.781250,52318000.0,27.375000,3.559040e+07
3,1980-12-17,25.875000,25.999999,25.875000,25.875000,21610400,0.382500,26.781250,52318000.0,27.375000,3.559040e+07
4,1980-12-18,26.625000,26.750000,26.625000,26.625000,18362400,0.393587,26.250000,27594000.0,27.375000,3.559040e+07
5,1980-12-19,28.249999,28.374999,28.249999,28.249999,12157600,0.417608,26.500000,19640600.0,27.375000,3.559040e+07
6,1980-12-22,29.625000,29.750000,29.625000,29.625000,9340800,0.437934,27.593750,15367800.0,27.375000,3.559040e+07
7,1980-12-23,30.874999,30.999999,30.874999,30.874999,11737600,0.456413,28.843750,12899600.0,27.678571,2.051600e+07
8,1980-12-24,32.500000,32.624999,32.500000,32.500000,12000800,0.480434,30.312500,11309200.0,28.428571,1.594880e+07
9,1980-12-26,35.500001,35.625001,35.500001,35.500001,13893600,0.524782,32.125000,11743200.0,29.892857,1.415760e+07


In [175]:
calc_bollinger(aapl, timespan, ["Close", "Volume"], merge_result=True, scaler=2)

,Date,Open,High,Low,Close,Volume,Adj Close,Close_roll_2std_4,Close_bollinger_4_up,Close_bollinger_4_low,Close_roll_2std_7,Close_bollinger_7_up,Close_bollinger_7_low,Volume_roll_2std_4,Volume_bollinger_4_up,Volume_bollinger_4_low,Volume_roll_2std_7,Volume_bollinger_7_up,Volume_bollinger_7_low
0,1980-12-12,28.750000,28.875000,28.750000,28.750000,117258400,0.425000,3.111638,31.861638,25.638362,3.168859,31.918859,25.581141,8.869398e+07,2.059524e+08,2.856442e+07,7.551670e+07,1.927751e+08,4.174170e+07
1,1980-12-15,27.375001,27.375001,27.250000,27.250000,43971200,0.402826,3.111638,30.361638,24.138362,3.168859,30.418859,24.081141,8.869398e+07,1.326652e+08,-4.472278e+07,7.551670e+07,1.194879e+08,-3.154550e+07
2,1980-12-16,25.375000,25.375000,25.250000,25.250000,26432000,0.373261,3.111638,28.361638,22.138362,3.168859,28.418859,22.081141,8.869398e+07,1.151260e+08,-6.226198e+07,7.551670e+07,1.019487e+08,-4.908470e+07
3,1980-12-17,25.875000,25.999999,25.875000,25.875000,21610400,0.382500,3.111638,28.986638,22.763362,3.168859,29.043859,22.706141,8.869398e+07,1.103044e+08,-6.708358e+07,7.551670e+07,9.712710e+07,-5.390630e+07
4,1980-12-18,26.625000,26.750000,26.625000,26.625000,18362400,0.393587,1.744037,28.369037,24.880963,3.168859,29.793859,23.456141,2.282072e+07,4.118312e+07,-4.458317e+06,7.551670e+07,9.387910e+07,-5.715430e+07
5,1980-12-19,28.249999,28.374999,28.249999,28.249999,12157600,0.417608,2.590044,30.840043,25.659955,3.168859,31.418858,25.081140,1.197956e+07,2.413716e+07,1.780446e+05,7.551670e+07,8.767430e+07,-6.335910e+07
6,1980-12-22,29.625000,29.750000,29.625000,29.625000,9340800,0.437934,3.356430,32.981430,26.268570,3.168859,32.793859,26.456141,1.122900e+07,2.056980e+07,-1.888198e+06,7.551670e+07,8.485750e+07,-6.617590e+07
7,1980-12-23,30.874999,30.999999,30.874999,30.874999,11737600,0.456413,3.653622,34.528621,27.221377,4.064216,34.939215,26.810783,7.694851e+06,1.943245e+07,4.042749e+06,2.398318e+07,3.572078e+07,-1.224558e+07
8,1980-12-24,32.500000,32.624999,32.500000,32.500000,12000800,0.480434,3.619968,36.119968,28.880032,5.409977,37.909977,27.090023,2.647318e+06,1.464812e+07,9.353482e+06,1.262633e+07,2.462713e+07,-6.255331e+05
9,1980-12-26,35.500001,35.625001,35.500001,35.500001,13893600,0.524782,5.078551,40.578552,30.421450,6.772230,42.272231,28.727771,3.734905e+06,1.762850e+07,1.015870e+07,8.602481e+06,2.249608e+07,5.291119e+06


In [176]:
sp500 = load_raw_frame("^GSPC")

In [177]:
sp500_i = sp500.set_index("Date")
aapl_i = aapl.set_index("Date")

In [178]:
joined_df = pd.merge(aapl_i.loc[:,["Close"]], sp500_i.loc[:,["Close"]], how="left", left_index=True, right_index=True)

In [180]:
beta_df = timewindow_alphabeta(aapl, sp500, ["Close", "Open"], 10, merge_result=False)
beta_df.head()

,timewindow_beta_10_Close,timewindow_alpha_10_Close,timewindow_beta_10_Open,timewindow_alpha_10_Open
Date,,,,
1980-12-12,0.0,0.0,0.0,0.0
1980-12-15,0.0,0.0,0.0,0.0
1980-12-16,0.0,0.0,0.0,0.0
1980-12-17,0.0,0.0,0.0,0.0
1980-12-18,0.0,0.0,0.0,0.0


In [181]:
beta_df

,timewindow_beta_10_Close,timewindow_alpha_10_Close,timewindow_beta_10_Open,timewindow_alpha_10_Open
Date,,,,
1980-12-12,0.000000,0.000000,0.000000,0.000000
1980-12-15,0.000000,0.000000,0.000000,0.000000
1980-12-16,0.000000,0.000000,0.000000,0.000000
1980-12-17,0.000000,0.000000,0.000000,0.000000
1980-12-18,0.000000,0.000000,0.000000,0.000000
1980-12-19,0.000000,0.000000,0.000000,0.000000
1980-12-22,0.000000,0.000000,0.000000,0.000000
1980-12-23,0.000000,0.000000,0.000000,0.000000
1980-12-24,0.000000,0.000000,0.000000,0.000000


In [182]:
beta_df

,timewindow_beta_10_Close,timewindow_alpha_10_Close,timewindow_beta_10_Open,timewindow_alpha_10_Open
Date,,,,
1980-12-12,0.000000,0.000000,0.000000,0.000000
1980-12-15,0.000000,0.000000,0.000000,0.000000
1980-12-16,0.000000,0.000000,0.000000,0.000000
1980-12-17,0.000000,0.000000,0.000000,0.000000
1980-12-18,0.000000,0.000000,0.000000,0.000000
1980-12-19,0.000000,0.000000,0.000000,0.000000
1980-12-22,0.000000,0.000000,0.000000,0.000000
1980-12-23,0.000000,0.000000,0.000000,0.000000
1980-12-24,0.000000,0.000000,0.000000,0.000000


In [183]:
pd.merge(aapl,beta_df,left_on='Date',right_index=True)

,Date,Open,High,Low,Close,Volume,Adj Close,timewindow_beta_10_Close,timewindow_alpha_10_Close,timewindow_beta_10_Open,timewindow_alpha_10_Open
0,1980-12-12,28.750000,28.875000,28.750000,28.750000,117258400,0.425000,0.000000,0.000000,0.000000,0.000000
1,1980-12-15,27.375001,27.375001,27.250000,27.250000,43971200,0.402826,0.000000,0.000000,0.000000,0.000000
2,1980-12-16,25.375000,25.375000,25.250000,25.250000,26432000,0.373261,0.000000,0.000000,0.000000,0.000000
3,1980-12-17,25.875000,25.999999,25.875000,25.875000,21610400,0.382500,0.000000,0.000000,0.000000,0.000000
4,1980-12-18,26.625000,26.750000,26.625000,26.625000,18362400,0.393587,0.000000,0.000000,0.000000,0.000000
5,1980-12-19,28.249999,28.374999,28.249999,28.249999,12157600,0.417608,0.000000,0.000000,0.000000,0.000000
6,1980-12-22,29.625000,29.750000,29.625000,29.625000,9340800,0.437934,0.000000,0.000000,0.000000,0.000000
7,1980-12-23,30.874999,30.999999,30.874999,30.874999,11737600,0.456413,0.000000,0.000000,0.000000,0.000000
8,1980-12-24,32.500000,32.624999,32.500000,32.500000,12000800,0.480434,0.000000,0.000000,0.000000,0.000000
9,1980-12-26,35.500001,35.625001,35.500001,35.500001,13893600,0.524782,0.602909,115.725489,0.677632,112.616838


In [ ]:
"""
1980-12-26     0.602909
1980-12-29     0.462633
1980-12-30     0.362929
1980-12-31     0.272490
1981-01-02     0.251707
1981-01-05     0.193142
"""

In [ ]:
_i = 10

s = joined_df.iloc[ 0:10, :]["Close_x"]
m = joined_df.iloc[ 0:10, :]["Close_y"]

slope, intercept, r_value, p_value, std_err = stats.linregress(s.values,m.values)
slope

In [ ]:
tier1_df = calc_measures_tier1(aapl)
tier1_df

In [202]:
tier1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9141 entries, 0 to 9140
Columns: 159 entries, Date to timewindow_alpha_20_Low
dtypes: float64(157), int64(1), object(1)
memory usage: 11.2+ MB


In [ ]:
store_tier1_frame(tier1_df, "AAPL")

In [ ]:
load_tier1_frame("AAPL")

In [ ]:
tier1_df = calc_measures_tier1(aapl, verbose=True)

In [ ]:
%%timeit
tier1_df = calc_measures_tier1(aapl, verbose=True)

In [ ]:
aapl.head()

In [ ]:
timewindow_alpha(aapl, sp500, ["Close","Volume","Low"], 10)

In [ ]:
timewindow_beta(aapl, sp500, ["Close","Volume","Low"], 10)

In [ ]:
timewindow_alphabeta(aapl, sp500, ["Close","Volume","Low"], 10)

In [ ]:
ticker_list = pd.read_csv("RES_initial_dataload.csv")
ticker_list = ticker_list[ticker_list["STATUS"] == "OK"].iloc[0]
ticker_list

for ix, row in ticker_list.iterrows():
    itr_ticker = row["Symbol"]
    print("\n\n - {} - \n".format(itr_ticker)
    itr_df = load_raw_frame("AAPL")
    tier1_df = calc_measures_tier1(itr_df, verbose=True)
    store_tier1_frame(tier1_df, itr_ticker)
    

In [ ]:
aapl = load_raw_frame("AAPL")
tier1_df = calc_measures_tier1(aapl, verbose=True)
store_tier1_frame(tier1_df, "APPL")

In [ ]:
tier1_df.head()

In [ ]:
# Open Close Diff             |   Open(t) - Close(t -1)
# Open Close Return           |   (Close(t -1) / Open(t)) - 1
# Open Close Return MA        |   SMA(Open Close Return)
# Open Close Return Bollinger |   Bollinger(Open Close Return)

In [ ]:
aapl.head()

In [ ]:
# Open Close Diff

aftermarketreturn = (aapl["Open"] - aapl["Close"].shift(1))
aftermarketreturn.head()

In [ ]:
# Open Close Return

aftermarketdiff = (aapl["Open"] / aapl["Close"].shift(1)) - 1
aftermarketdiff.head()

In [199]:
calc_aftermarket_bollinger??

In [ ]:
# Open Close Diff             |   Open(t) - Close(t -1)
calc_aftermarket_diff(aapl).head()

In [ ]:
# Open Close Return           |   (Close(t -1) / Open(t)) - 1
calc_aftermarket_return(aapl).head()

In [ ]:
not "D" in aapl.columns

In [ ]:
# Open Close Return MA        |   SMA(Open Close Return)
calc_aftermarket_sma(aapl, timespan)

In [ ]:
# Open Close Return Bollinger |   Bollinger(Open Close Return)
calc_aftermarket_bollinger(aapl, timespan)

In [ ]:
aapl_t1 = load_tier1_frame("AAPL")

In [ ]:
aapl_t1.columns.tolist()

In [ ]:
returns_cols = [x for x in aapl_t1.columns.tolist() if "timewindow_return_" in x]
returns_cols

In [ ]:
returns_cols_disc = ["discrete_{}".format(x) for x in returns_cols]

In [ ]:
data = np.random.random(100)
digitized = np.digitize(data, bins)
bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]


In [ ]:
x = np.array(np.linspace(0, 0.9, 10))
bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])
inds = np.digitize(x, bins)


for n in range(x.size):
    print(bins[inds[n]-1], "<=", x[n], "<", bins[inds[n]])
    


In [ ]:
x = np.array(np.linspace(0, 0.9, 10))
bins = np.array([0.0, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])
inds = np.digitize(x, bins)


for n in range(x.size):
    print(bins[inds[n]-1], "<=", x[n], "<", bins[inds[n]])
    


In [ ]:
x = np.array(np.linspace(-1.9, 0.0, 10))
bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.0, 0.001])
inds = np.digitize(x, bins)


for n in range(x.size):
    print(bins[inds[n]-1], "<=", x[n], "<", bins[inds[n]])
    


In [ ]:
x = np.array(np.linspace(-2.1, 1.0, 10))
bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])
inds = np.digitize(x, bins)
inds

In [ ]:
np.clip(inds, 0, bins.shape[0] - 1)

In [ ]:
x = aapl_t1.loc[ : , returns_cols].fillna(0).values
bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])
inds = np.digitize(x, bins, right=False)
inds = np.clip(inds, 0, bins.shape[0] - 1)

discretized_df = pd.DataFrame(inds, columns=returns_cols_disc)
discretized_df.head()

In [ ]:
pd.DataFrame(bins, columns=["Values"])

In [ ]:
loss_bins = np.array([-2.0, -1.0, -0.5, 0.0])
gain_bins = np.array([0.0, 0.05, 1.0, 2.0])

In [ ]:
loss_bins

In [ ]:
gain_bins

In [ ]:
x = np.linspace(-2.5, 2.5, 7)
x

In [ ]:
inds_loss = np.digitize(x, loss_bins, right=False)
inds_loss = np.clip(inds_loss, 0, loss_bins.shape[0] - 1)
inds_loss

In [ ]:
inds_gain = np.digitize(x, gain_bins)
inds_gain = np.clip(inds_gain, 0, gain_bins.shape[0] - 1)
inds_gain

In [ ]:
bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.0, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])

In [ ]:
def discretize(bins, return_index, v):
    _r = None
    if v <= 0:
        inds_gain = np.digitize(v, bins, right=True)
        inds_gain = np.clip(inds_gain, 0, bins.shape[0] - 1)

        _r = inds_gain
    else:
        inds_gain = np.digitize(v, bins, right=False) - 1
        inds_gain = np.clip(inds_gain, 0, bins.shape[0] - 1)

        _r = inds_gain
    
    if not ( return_index ):
        _r = bins[_r]
    
    return _r

_bins = np.array([-2.0, -1.0, -0.8, -0.5, -0.3, -0.1, -0.05, 0.0, 0.01, 0.05, 0.07, 0.1, 0.2, 0.4, 1.0])
_disc = partial(discretize, _bins, False)

_real = [-3, -2, -1, -0.7, -0.5, 0, 0.005, 0.01, 0.012, 0.04, 0.05, 0.06, 0.08, 0.1, 0.15, 0.2, 0.3, 0.4, 0.9, 1, 1.2, 40]
_discrete = pd.DataFrame(_real, index=_real).applymap(_disc)



In [ ]:
x = aapl_t1.loc[ : , returns_cols].fillna(0)

discretize_df = x.applymap(discretize)
discretized_df = pd.DataFrame(discretize_df, columns=returns_cols_disc)
discretized_df

In [208]:
tier1_df = calc_measures_tier1(aapl)
print(tier1_df.columns.shape)
tier1_df = calc_measures_tier2(tier1_df)
print(tier1_df.columns.shape)

| | START     - 2017-03-15 16:15:44.192986
\ / SMA       - 0:00:00.086125
\ / RETURNS   - 0:00:00.092693
\ / DIFF MOVE - 0:00:00.096596
\ / BOLLINGER - 0:00:00.117711


/opt/conda/lib/python3.5/site-packages/scipy/stats/_stats_mstats_common.py:95: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/opt/conda/lib/python3.5/site-packages/scipy/stats/_stats_mstats_common.py:97: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


\ / ALPHABETA - 0:02:26.536868
 V  END       - 2017-03-15 16:18:11.122979 (TOOK 0:02:26.929993)
(161,)
| | START     - 2017-03-15 16:18:11.123713
\ / AM DIFF       - 0:00:00.001957
\ / AM RETURNS   - 0:00:00.001845
\ / AM SMA - 0:00:00.047715
\ / AM BOLLINGER - 0:00:00.065953
 V  END       - 2017-03-15 16:18:11.241268 (TOOK 0:00:00.117555)
(184,)


In [31]:
aapl = load_raw_frame("AAPL")
aapl["Date"].head()

0   1980-12-12
1   1980-12-15
2   1980-12-16
3   1980-12-17
4   1980-12-18
Name: Date, dtype: datetime64[ns]

In [44]:
from dateutil import parser as dtparser

In [50]:
(aapl["Date"] > dtparser.parse("12-12-1980")) & (aapl["Date"] <= dtparser.parse("16-12-1980"))

0       False
1        True
2        True
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
9112    False
9113    False
9114    False
9115    False
9116    False
9117    False
9118    False
9119    False
9120    False
9121    False
9122    False
9123    False
9124    False
9125    False
9126    False
9127    False
9128    False
9129    False
9130    False
9131    False
9132    False
9133    False
9134    False
9135    False
9136    False
9137    False
9138    False
9139    False
9140    False
9141    False
Name: Date, dtype: bool